In [1]:
import numpy as np

def calculate_element_local_stiffness_matrix(E, A, L):
    ke_local = np.zeros((4,4))
    ke_local[0, 0] = E * A / L
    ke_local[0, 2] = -E * A / L
    ke_local[2, 0] = -E * A / L
    ke_local[2, 2] = E * A / L
    return ke_local

def define_element_rotation_matrix(theta):
    beta = np.zeros((4, 4))
    unit_rotation = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
    beta[0:2, 0:2] = unit_rotation
    beta[2:4, 2:4] = unit_rotation
    return beta

def calculate_element_length(node_i_coordinates, node_j_coordinates):
    L = np.linalg.norm(node_j_coordinates - node_i_coordinates)
    return L

def calculate_element_orientation(node_i_coordinates, node_j_coordinates):
    delta = node_j_coordinates - node_i_coordinates
    theta = np.arctan2(delta[1], delta[0])
    return theta

def define_global_element_stiffness_matrix(ke_local, beta):
    ke_global = np.dot(np.dot(np.transpose(beta), ke_local), beta)
    return ke_global

def define_global_dof_for_element(node_number_i, node_number_j):
    dof_i = [node_number_i * 2 - 2, node_number_i * 2 - 1]
    dof_j = [node_number_j * 2 - 2, node_number_j * 2 - 1]
    dof = dof_i + dof_j
    return dof

def assemble_global_stiffness_matrix(num_dof, ke_global, global_dof):
    K = np.zeros((num_dof, num_dof))
    for i in range(len(ke_global)):
        K[global_dof[i][0]:(global_dof[i][1] + 1), global_dof[i][0]:(global_dof[i][1] + 1)] += ke_global[i][0:2, 0:2]
        K[global_dof[i][0]:(global_dof[i][1] + 1), global_dof[i][2]:(global_dof[i][3] + 1)] += ke_global[i][0:2, 2:4]
        K[global_dof[i][2]:(global_dof[i][3] + 1), global_dof[i][0]:(global_dof[i][1] + 1)] += ke_global[i][2:4, 0:2]
        K[global_dof[i][2]:(global_dof[i][3] + 1), global_dof[i][2]:(global_dof[i][3] + 1)] += ke_global[i][2:4, 2:4]
    return K

In [2]:
import pandas as pd

node_data = pd.read_csv(r"C:\Users\madis\Downloads\nodes.csv")
element_data = pd.read_csv(r"C:\Users\madis\Downloads\elements.csv")

nodes = []
for i in range(len(node_data)):
    nodes.append([node_data.X[i], node_data.Y[i]])
    
material_properties = [29500000.0]
section_properties = [0.877, 0.30]

elements = [[element_data.node_i[i], element_data.node_j[i], element_data.mat_prop[i], element_data.sect_prop[i]] for i in range(len(element_data))]

s = [0, 1, 23]

u = np.zeros(2 * len(nodes))

L = [calculate_element_length(np.array(nodes[elements[i][0] - 1]), np.array(nodes[elements[i][1] - 1])) for i in range(len(elements))] 
w = 229.84 / 12

F = np.zeros(len(nodes) * 2)

top_chord_elements = range(1, 12)

for i in top_chord_elements:
    F[2 * i - 1] += w * L[i - 1] / 2
    F[2 * i + 1] += w * L[i - 1] / 2
    
A = [section_properties[elements[i][3] - 1] for i in range(len(elements))]
E = [material_properties[elements[i][2] - 1] for i in range(len(elements))]
theta = [calculate_element_orientation(np.array(nodes[elements[i][0] - 1]), np.array(nodes[elements[i][1] - 1])) for i in range(len(elements))]

ke_local = [calculate_element_local_stiffness_matrix(E[i], A[i], L[i]) for i in range(len(L))]

beta = [define_element_rotation_matrix(theta[i]) for i in range(len(theta))]

ke_global = [define_global_element_stiffness_matrix(ke_local[i], beta[i]) for i in range(len(ke_local))]

element_global_dof = [define_global_dof_for_element(elements[i][0], elements[i][1]) for i in range(len(elements))]

num_dof = len(nodes) * 2

K = assemble_global_stiffness_matrix(num_dof, ke_global, element_global_dof)

p = np.setdiff1d(np.arange(num_dof), s)

Kpp = K[np.ix_(p, p)]
Kss = K[np.ix_(s, s)]
Kps = K[np.ix_(p, s)]
Ksp = K[np.ix_(s, p)]

Fp = F[p]
us = u[s]

up = np.dot(np.linalg.inv(Kpp), (Fp - np.dot(Kps, us)))
u[p] = up
Fs = np.dot(Ksp, up) + np.dot(Kss, us)

u_element_global = [u[element_global_dof[i]] for i in range(len(element_global_dof))]
delta = [np.dot(beta[i], u_element_global[i]) for i in range(len(u_element_global))]
P = [np.dot(ke_local[i], delta[i]) for i in range(len(ke_local))]

print("Maximum Displacement:")
print(np.max(u_element_global))
print()
print("Internal Forces in Element 9")
print(P[8])
print()
print("Global Displacements:")
print(u_element_global)
print()
print("Local Displacements:")
print(delta)
print()
print("Internal Forces:")
print(P)

Maximum Displacement:
1.9040694636638746

Internal Forces in Element 9
[-17191.75317858      0.          17191.75317858      0.        ]

Global Displacements:
[array([0.        , 0.        , 0.00979271, 0.64427537]), array([0.00979271, 0.64427537, 0.0291543 , 1.09942035]), array([0.0291543 , 1.09942035, 0.05746923, 1.47042666]), array([0.05746923, 1.47042666, 0.09234994, 1.73640701]), array([0.09234994, 1.73640701, 0.13140887, 1.88315804]), array([0.13140887, 1.88315804, 0.17225847, 1.90316033]), array([0.17225847, 1.90316033, 0.21251118, 1.79557838]), array([0.21251118, 1.79557838, 0.24977944, 1.56626065]), array([0.24977944, 1.56626065, 0.28167571, 1.22773949]), array([0.28167571, 1.22773949, 0.30581241, 0.79923123]), array([0.30581241, 0.79923123, 0.32646104, 0.        ]), array([0.32297822, 0.32642121, 0.30730988, 0.82161185]), array([0.30730988, 0.82161185, 0.28179287, 1.2480681 ]), array([0.28179287, 1.2480681 , 0.24881475, 1.58176967]), array([0.24881475, 1.58176967, 0.21076307